In [1]:
import pandas as pd

# Leitura da Base IPDM (Índice Paulistas de Desenvolvimento Municipal)
ipdm = pd.read_csv("arq_ipdm_painel_v2024.csv",
                   sep=';',
                   encoding='ISO-8859-1')

In [2]:
ipdm_pos_process = pd.DataFrame()

df_temp = ipdm.copy()
df_temp['Valor'] = df_temp['Valor'].str.replace(',', '.', regex=False).astype(float, errors='ignore')

for tipo in df_temp['Tipo'].unique():
        df_pivot = df_temp[df_temp['Tipo'] == tipo].pivot_table(
            
            index = ['cod_ibge', 'Municipio', 'Ano'],
            columns=['Indicador5', 'Indicador4', 'Indicador3', 'Indicador2', 'Indicador1'],
            values='Valor',
            aggfunc='first'
            
        ).reset_index()
        
        if tipo == 'IPDM':
            df_pivot.columns = ['cod_ibge', 'Municipio', 'Ano',
                                'IPDM']
            
        elif tipo == 'Escolaridade':
            df_pivot.columns = ['cod_ibge', 'Municipio', 'Ano',
                                'Taxa de atendimento escolar na faixa etária de 0 a 3 anos',
                                'Proporção média de alunos do 5º ano com proficiência em Língua Portuguesa e Matemática',
                                'Proporção média de alunos do 9º ano com proficiência em Língua Portuguesa e Matemática',
                                'Taxas de distorção idade-série no Ensino Médio',
                                'Indicador Escolaridade']
            
        elif tipo == 'Riqueza':
            df_pivot.columns = ['cod_ibge', 'Municipio', 'Ano', 
                        'Consumo anual de energia elétrica residencial', 
                        'Consumo anual de energia elétrica comercial', 
                        'Rendimento do trabalho formal', 
                        'Produto Interno Bruto per capita', 
                        'Indicador Riqueza']

        elif tipo == 'Longevidade':
            df_pivot.columns = ['cod_ibge', 'Municipio', 'Ano',
                                'Taxas de mortalidade infantil',
                                'Taxas de mortalidade perinatal',
                                'Taxas de mortalidade 15 a 39 anos',
                                'Taxas de mortalidade 60 a 69 anos',
                                'Indicador Longevidade']
            
        if ipdm_pos_process.empty:
            ipdm_pos_process = df_pivot

        else:
            
            ipdm_pos_process = ipdm_pos_process.merge(df_pivot,
                                                      how='inner', on=['cod_ibge', 'Municipio', 'Ano'])
            

ipdm_pos_process = ipdm_pos_process.drop_duplicates()

In [3]:
# Verificação se todos os municípios foram devidamente processados
len(ipdm_pos_process['Municipio'].unique())

645

In [4]:
# Base do Município de Adamantina após a formatação devida
ipdm_pos_process[ipdm_pos_process['Municipio'] == 'Adamantina']

,cod_ibge,Municipio,Ano,IPDM,Consumo anual de energia elétrica residencial,Consumo anual de energia elétrica comercial,Rendimento do trabalho formal,Produto Interno Bruto per capita,Indicador Riqueza,Taxas de mortalidade infantil,Taxas de mortalidade perinatal,Taxas de mortalidade 15 a 39 anos,Taxas de mortalidade 60 a 69 anos,Indicador Longevidade,Taxa de atendimento escolar na faixa etária de 0 a 3 anos,Proporção média de alunos do 5º ano com proficiência em Língua Portuguesa e Matemática,Proporção média de alunos do 9º ano com proficiência em Língua Portuguesa e Matemática,Taxas de distorção idade-série no Ensino Médio,Indicador Escolaridade
0,3500105,Adamantina,2014,0.548,2.420003,8.975414,1375.205579,53854.100878,0.365,7.421150,11.059908,1.013820,12.879959,0.770,43.394935,49.796088,26.881342,8.444444,0.510
1,3500105,Adamantina,2016,0.554,2.291891,8.801709,1413.171805,61154.621155,0.365,12.855831,14.545455,1.062092,12.967201,0.710,48.259386,69.060852,26.574696,7.142857,0.587
2,3500105,Adamantina,2018,0.560,2.641364,9.591297,1556.704707,53854.100878,0.382,10.810811,19.538188,1.044760,12.827663,0.686,51.528074,72.893672,28.064909,7.511046,0.612
3,3500105,Adamantina,2020,0.594,2.728212,8.823260,1489.333082,52045.297886,0.378,3.384095,12.573345,1.065142,12.912355,0.779,53.658537,72.977918,29.286725,6.623932,0.625
4,3500105,Adamantina,2022,0.565,2.452972,8.324105,1584.058333,41516.103919,0.359,8.802817,11.373578,1.177045,14.683544,0.742,54.559387,60.285000,31.085000,5.690000,0.594


In [6]:
ipdm_pos_process.to_csv("ipdm_processada.csv")